Let's do some analyses combining trees and map objects

In [ ]:
using Phylo # phylogenetics
using SpatialEcology #spatial ecology, duh
using Plots # plotting
using JLD2, SparseArrays, DataFrames #jld2 is to load our files. Due to a bug we need the other two

#### Loading and curating the data

In [ ]:
@load "African_mammals.jld"

Load the tree

In [ ]:
tree = open(t->parsenewick(t, NamedPolytomousTree), "../Data/Mammals.tree")

We always start with a plot - it'll take a little time

In [ ]:
plot(tree, treetype = :fan, tipfont = (2,))

We want to align the tree and the dataset. We do that by dropping all species that don't overlap. It becomes a little more tricky as there is used an underscore in the names in the tree

In [ ]:
names = getleafnames(tree)

In [ ]:
spnames = replace.(speciesnames(mammals), Ref(" "=>"_"))

In [ ]:
not_in = setdiff(names, spnames)

In [ ]:
droptips!(tree, not_in)

Here is the new tree

In [ ]:
plot(tree, treetype = :fan, tipfont = (2,), fmt = :png)

#### Node-based analysis
We want to use this new tree to compare the richness of monophyletic sister groups, which is interesting to see the spatial signature of evolution

Find the names of all internal nodes

In [ ]:
collect(nodenamefilter(!isleaf, tree))

Let's pick a random node, 791, and find all species descending from that

In [ ]:
desc = getdescendants(tree, "Node 791")
ds = filter(x->isleaf(tree, x), desc)

We'll make a view containing only these species

In [ ]:
n791 = view(mammals, species = replace.(ds, Ref("_"=>" ")))
plot(n791)

Now this was interesting, we'd want to be able to do this many times. So let's write a function to do it

In [ ]:
getnode(nodenumber::Int, asm) = getnode("Node $nodenumber", asm)
function getnode(nodename, asm)
    desc = getdescendants(tree, nodename)
    ds = filter(x->isleaf(tree, x), desc)
    view(asm, species = replace.(ds, Ref("_"=>" ")))
end

This will call the function on random clades and show their richness

In [ ]:
randnode = rand(collect(nodenamefilter(!isleaf, tree)))
n = getnode(randnode, mammals)
plot(n, title = randnode)

Now, let's compare the richness of the two clades descending from that node

In [ ]:
gc = getchildren(tree, "Node 791")

In [ ]:
ch = getnode.(gc, mammals)

In [ ]:
plot(plot.(ch)..., title = permutedims(gc))

We can see that they look quite different. So something interesting must have happened at this point in history. But is the difference significant from a random expectation? Let's run a null model on this.
#### Randomizing occurrences for null model analysis

In [ ]:
using RandomBooleanMatrices, Random

Gives us a method for randomizing matrices while maintaining marginal totals

In [ ]:
m = matrixrandomizer(occurrences(n791))
plot(
    heatmap(rand!(m)),
    heatmap(rand!(m))
)

Due to Julia's dispatch system, we also have a method to randomize our occurrences data sets

In [ ]:
m = matrixrandomizer(n791)

In [ ]:
randm = rand!(m)
plot(randm)

In [ ]:
# Empirical species richness of the two daughter nodes
plot( plot.(getnode.(gc, Ref(mammals)))..., title = permutedims(gc))

In [ ]:
# Random species richness of the two daughter nodes
plot( plot.(getnode.(gc, Ref(randm)))..., title = permutedims(gc))

Let's get the empirical richness of one of the clades

In [ ]:
dec2 = getnode(790, mammals)
emprich = collect(richness(dec2))

Now let's do a 100 randomizations of the total clade. This is slow because the `getnode` function is slow (will be much faster in the future)

In [ ]:
randrich = [collect(richness(getnode(790, rand!(m)))) for i in 1:100]

We now have an array of 100 richness values. Let's calculate the mean and standard deviation, and use this to calculate standardized effect size

In [ ]:
using Statistics
temp = reduce(hcat,randrich)
sdt, meant = vec(mapslices(std, temp, dims = 2)), vec(mean(temp, dims = 2))

In [ ]:
z = (emprich .- meant) ./ sdt

A plot tells us that deviations are much larger than the ~2 expected from random chance in certain areas

In [ ]:
plot(z, n791, clim = (-5,5), c = :RdYlBu_r)